In [238]:
import pandas as pd
import requests
import re

# Yelp Data

### Extract CSVs into DataFrames

In [239]:
yelp_file = "../Resources/yelp_data.csv"
yelp_df = pd.read_csv(yelp_file)
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",NaN
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",NaN
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",NaN
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [240]:
# All columns
yelp_df.columns

Index(['restaurant_name', 'yelp_url', 'restaurant_rating', 'price_cuisine',
       'address1', 'address2', 'restaurant_website'],
      dtype='object')

In [241]:
# restaurant_name check
yelp_df["restaurant_name"].value_counts()


Sandwiches Unlimited Lunch Box    2
Patrick’s Pub                     1
The Second Half On Main           1
Knotty Pine Pub                   1
Miga Sushi                        1
                                 ..
Stirling Tavern                   1
Sheimer’s Deli                    1
Bollywood Grill 2                 1
El Lechon De Negron               1
Outback Steakhouse                1
Name: restaurant_name, Length: 237, dtype: int64

### Dropping duplicate restaurant_name and address1 value

In [242]:
# clear leading and trailing spaces
yelp_df['restaurant_name'] = yelp_df['restaurant_name'].str.strip()
yelp_df['address1'] = yelp_df['address1'].str.strip()
yelp_df['address2'] = yelp_df['address2'].str.strip()

# drop duplicates for restaurant name and address
yelp_df = yelp_df.drop_duplicates(subset=['restaurant_name', 'address1','address2'])
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",NaN
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",NaN
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",NaN
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [243]:
# average rating
average_rating = yelp_df["restaurant_rating"].sum()/yelp_df["restaurant_name"].count()
print("average_rating")
print(average_rating)


average_rating
3.9936974789915967


In [244]:
# fill empty cells
yelp_df=yelp_df.fillna("")
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [245]:
# restaurant_rating check
yelp_df["restaurant_rating"].count()

238

### Clear restaurant website if it does not exist

In [246]:
# restaurant_url check
yelp_df["restaurant_website"].value_counts()

                                   52
Find a Table                        3
anthonyandsonsbakery.com            2
sandwichesunlimitedlunchbox.co…     2
jaialairestaurant.com               1
                                   ..
rockawayriverbarn.com               1
thelotusthai.com                    1
luzzishiberniainn.com               1
stirlingtavern.com                  1
sergioandco.com                     1
Name: restaurant_website, Length: 183, dtype: int64

In [247]:
# check for valid website
for index, row in yelp_df.iterrows():
    if ('.' not in row['restaurant_website']) | (' ' in row['restaurant_website']):
        yelp_df.loc[index, 'restaurant_website'] = ""
    
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [248]:
# restaurant_website check
yelp_df["restaurant_website"].value_counts()

                                   55
sandwichesunlimitedlunchbox.co…     2
anthonyandsonsbakery.com            2
OliveGarden.com                     1
tacoria.com                         1
                                   ..
rivieramayanj.com                   1
TheodorasRestaurantPizza.com        1
rockawayriverbarn.com               1
thelotusthai.com                    1
olivelucys.com                      1
Name: restaurant_website, Length: 182, dtype: int64

### Split the price_cuisine column in to price_level and cuisine column

In [249]:
# price_cuisine check
yelp_df["price_cuisine"].value_counts()

$$Italian                                           12
$$Thai                                               6
$Chinese                                             5
$$Indian                                             5
$$Mexican                                            5
                                                    ..
Caribbean                                            1
$$Barbeque, American (Traditional), Comfort Food     1
$$American (Traditional), Seafood, Italian           1
Soul Food                                            1
$Delis, Sandwiches                                   1
Name: price_cuisine, Length: 183, dtype: int64

In [250]:
price_level = []
cuisine = []
for index, row in yelp_df.iterrows():
    price_cuisine = re.split(r'(\$+)', row['price_cuisine'])
    if len(price_cuisine) == 1:
        price_level.append("")
        cuisine.append(price_cuisine[0])
    else:
        price_level.append(price_cuisine[1])
        cuisine.append(price_cuisine[2])
yelp_df['price_level'] = price_level
yelp_df['cuisine'] = cuisine
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website,price_level,cuisine
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",,$$$,"Italian, Bars, Venues & Event Spaces"
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com,$$,"American (New), Bars, Pizza"
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com,$$,"Wine Bars, French, Italian"
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates"
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com,$$,Italian
...,...,...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",,$,"Food Trucks, American (Traditional), Hot Dogs"
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",,$,Pizza
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",,$,"Juice Bars & Smoothies, Soup, Sandwiches"
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…,,"Juice Bars & Smoothies, Acai Bowls, Wraps"


### Split address2 to city, state and zip code

In [251]:
# split with comma as delimiter to get city and state + zipcode
yelp_df[['city','state+zipcode']]=yelp_df.address2.str.split(",",expand=True,)
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website,price_level,cuisine,city,state+zipcode
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ 07869
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ 07869
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ 07885
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ 07801
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com,$$,Italian,Randolph,NJ 07869
...,...,...,...,...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",,$,"Food Trucks, American (Traditional), Hot Dogs",Dover,NJ 07801
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",,$,Pizza,Succasunna,NJ 07876
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",,$,"Juice Bars & Smoothies, Soup, Sandwiches",Dover,NJ 07801
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…,,"Juice Bars & Smoothies, Acai Bowls, Wraps",Morristown,NJ 07960


In [252]:
# clear leading spaces
yelp_df["state+zipcode"]=yelp_df["state+zipcode"].str.strip()

In [253]:
# split with space as delimiter to get state and zip code
yelp_df[['state', 'zip code']] = yelp_df["state+zipcode"].str.split(" ",expand=True)
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website,price_level,cuisine,city,state+zipcode,state,zip code
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ 07869,NJ,07869
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ 07869,NJ,07869
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ 07885,NJ,07885
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ 07801,NJ,07801
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com,$$,Italian,Randolph,NJ 07869,NJ,07869
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",,$,"Food Trucks, American (Traditional), Hot Dogs",Dover,NJ 07801,NJ,07801
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",,$,Pizza,Succasunna,NJ 07876,NJ,07876
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",,$,"Juice Bars & Smoothies, Soup, Sandwiches",Dover,NJ 07801,NJ,07801
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…,,"Juice Bars & Smoothies, Acai Bowls, Wraps",Morristown,NJ 07960,NJ,07960


### Clean Yelp DataFrame to have only Zip Codes of Morris County.

In [254]:
# retrieve all Morris County zip codes and add them to morris_county_zipcodes list
tables=pd.read_html('http://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=nj&StName=&StFIPS=&FIPS=34027')
morris_county_df = tables[2]
morris_county_df['Zip Code'] = morris_county_df['Zip Code'].astype(str)
morris_county_zipcodes = morris_county_df['Zip Code'].str.zfill(5)
morris_county_zipcodes

0     07005
1     07034
2     07035
3     07045
4     07046
5     07054
6     07058
7     07082
8     07405
9     07405
10    07440
11    07444
12    07457
13    07801
14    07802
15    07803
16    07803
17    07806
18    07806
19    07828
20    07834
21    07836
22    07842
23    07845
24    07847
25    07849
26    07850
27    07852
28    07853
29    07856
30    07857
31    07866
32    07869
33    07869
34    07870
35    07876
36    07878
37    07885
38    07926
39    07927
40    07928
41    07930
42    07932
43    07933
44    07935
45    07936
46    07940
47    07945
48    07946
49    07950
50    07950
51    07960
Name: Zip Code, dtype: object

In [255]:
# keep those restaurants that are in Morris County
yelp_df = yelp_df[yelp_df['zip code'].isin(morris_county_zipcodes)]
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website,price_level,cuisine,city,state+zipcode,state,zip code
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ 07869,NJ,07869
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ 07869,NJ,07869
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ 07885,NJ,07885
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ 07801,NJ,07801
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com,$$,Italian,Randolph,NJ 07869,NJ,07869
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",,$,"Food Trucks, American (Traditional), Hot Dogs",Dover,NJ 07801,NJ,07801
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",,$,Pizza,Succasunna,NJ 07876,NJ,07876
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",,$,"Juice Bars & Smoothies, Soup, Sandwiches",Dover,NJ 07801,NJ,07801
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…,,"Juice Bars & Smoothies, Acai Bowls, Wraps",Morristown,NJ 07960,NJ,07960


In [256]:
# drop unwanted columns and rename columns
yelp_df = yelp_df.drop(columns=['price_cuisine', 'address2', 'state+zipcode'])
yelp_df = yelp_df.rename(columns={'restaurant_rating':'yelp_rating',
                                  'price_level':'yelp_price_level',
                                  'cuisine':'yelp_cuisine'
                                 })
yelp_df['restaurant_name'] = yelp_df['restaurant_name'].str.strip()
yelp_df['address1'] = yelp_df['address1'].str.strip()

yelp_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city,state,zip code
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,1181 Sussex Tpke,,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ,07869
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,500 NJ-10,sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ,07869
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,322 S Main St,4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ,07885
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,64 E Mcfarlan St,quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ,07801
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,1171 Sussex Tpke,veronarestaurant.com,$$,Italian,Randolph,NJ,07869
...,...,...,...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,Dover Train Station,,$,"Food Trucks, American (Traditional), Hot Dogs",Dover,NJ,07801
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,275 State Rt 10 E,,$,Pizza,Succasunna,NJ,07876
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,6B S Warren St,,$,"Juice Bars & Smoothies, Soup, Sandwiches",Dover,NJ,07801
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,68 South St,cleanjuice.com/locations/morri…,,"Juice Bars & Smoothies, Acai Bowls, Wraps",Morristown,NJ,07960


# Google data

### Store CSV data into a DataFrame

In [257]:
csv_file = "../Resources/google_data.csv"
google_data_df = pd.read_csv(csv_file)
google_data_df

,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website
0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...
1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...
2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...
3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/
4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...
...,...,...,...,...,...,...,...
355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/
356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/
357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...
358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...


In [258]:
# All columns
google_data_df.columns

Index(['restaurant_name', 'place_id', 'price_level', 'restaurant_rating',
       'google_maps_url', 'address', 'restaurant_website'],
      dtype='object')

In [259]:
# restaurant name check
google_data_df["restaurant_name"].value_counts()

Subway                    12
Dunkin'                   11
Starbucks                 10
Chipotle Mexican Grill     4
Wendy's                    4
                          ..
Keo Ku Restaurant          1
Black Forest Inn           1
The Clay Oven              1
Pollos Pucalor             1
Black River Barn           1
Name: restaurant_name, Length: 245, dtype: int64

### Dropping duplicate restaurant address and restaurant_name value

In [260]:
# clear leading and trailing spaces
google_data_df['restaurant_name'] = google_data_df['restaurant_name'].str.strip()
google_data_df['address'] = google_data_df['address'].str.strip()

# drop duplicates for restaurant name and address
google_data_df = google_data_df.drop_duplicates(subset=['restaurant_name', 'address'])
google_data_df

,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website
0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...
1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...
2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...
3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/
4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...
...,...,...,...,...,...,...,...
355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/
356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/
357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...
358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...


### Clean DataFrame

In [261]:
# price_level check
google_data_df["price_level"].value_counts()

2.0    134
1.0     78
3.0     21
4.0      2
Name: price_level, dtype: int64

In [262]:
# restaurant_rating check
google_data_df["restaurant_rating"].value_counts()

4.2    47
4.3    44
4.5    39
4.4    39
4.1    26
4.6    20
4.0    14
4.7    13
3.9     6
3.4     5
3.8     5
3.7     5
3.6     4
4.8     3
5.0     3
3.3     2
3.5     2
2.5     2
2.9     1
3.2     1
3.0     1
2.0     1
3.1     1
Name: restaurant_rating, dtype: int64

In [263]:
# fill empty cells
google_data_df=google_data_df.fillna("")
google_data_df

,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website
0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...
1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...
2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...
3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/
4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...
...,...,...,...,...,...,...,...
355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/
356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/
357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...
358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...


### Split the Split address to address, city, state, zip code and country

In [264]:
# split from right with comma as delimiter
google_data_df[['address1','city','state_zip','country']] = google_data_df['address'].str.rsplit(',', 3, expand=True)
google_data_df['state_zip'] = google_data_df['state_zip'].str.lstrip()
google_data_df[['state','zip code']] = google_data_df['state_zip'].str.split(' ', 1, expand=True)
google_data_df

,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website,address1,city,state_zip,country,state,zip code
0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,Rockaway,NJ 07866,USA,NJ,07866
1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...,343 Mt Hope Ave,Rockaway,NJ 07866,USA,NJ,07866
2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...,301 Mt Hope Ave Suite 1048,Rockaway,NJ 07866,USA,NJ,07866
3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/,510 Tabor Rd,Morris Plains,NJ 07950,USA,NJ,07950
4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...,559 E Main St Store # 2,Denville,NJ 07834,USA,NJ,07834
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/,15 NJ-10,Succasunna,NJ 07876,USA,NJ,07876
356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/,312 Howard Blvd,Mt Arlington,NJ 07856,USA,NJ,07856
357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...,274 US-46,Mine Hill Township,NJ 07803,USA,NJ,07803
358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...,274 Rte 46,Mine Hill Township,NJ 07803,USA,NJ,07803


### Clean Google DataFrame to have only Zip Codes of Morris County.

In [265]:
google_data_df = google_data_df[google_data_df['zip code'].isin(morris_county_zipcodes)]
google_data_df

,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website,address1,city,state_zip,country,state,zip code
0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,Rockaway,NJ 07866,USA,NJ,07866
1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...,343 Mt Hope Ave,Rockaway,NJ 07866,USA,NJ,07866
2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...,301 Mt Hope Ave Suite 1048,Rockaway,NJ 07866,USA,NJ,07866
3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/,510 Tabor Rd,Morris Plains,NJ 07950,USA,NJ,07950
4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...,559 E Main St Store # 2,Denville,NJ 07834,USA,NJ,07834
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/,15 NJ-10,Succasunna,NJ 07876,USA,NJ,07876
356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/,312 Howard Blvd,Mt Arlington,NJ 07856,USA,NJ,07856
357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...,274 US-46,Mine Hill Township,NJ 07803,USA,NJ,07803
358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...,274 Rte 46,Mine Hill Township,NJ 07803,USA,NJ,07803


In [266]:
# Drop unwanted columns and rename columns
google_data_df = google_data_df.drop(columns=['place_id', 'address', 'country', 'state_zip'])
google_data_df = google_data_df.rename(columns={'restaurant_rating':'google_rating',
                                  'price_level':'google_price_level'
                                 })
google_data_df['restaurant_name'] = google_data_df['restaurant_name'].str.strip()
google_data_df['address1'] = google_data_df['address1'].str.strip()

google_data_df

,restaurant_name,google_price_level,google_rating,google_maps_url,restaurant_website,address1,city,state,zip code
0,Popeyes Louisiana Kitchen,1,3.7,https://maps.google.com/?cid=15487458085899557598,https://www.popeyes.com/store-locator/store/re...,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,Rockaway,NJ,07866
1,Buffalo Wild Wings,2,4.2,https://maps.google.com/?cid=8506076560359997359,https://www.buffalowildwings.com/en/locations/...,343 Mt Hope Ave,Rockaway,NJ,07866
2,Auntie Anne's,1,4.2,https://maps.google.com/?cid=11876493694413647283,https://www.auntieannes.com/nj/rockaway/119-1?...,301 Mt Hope Ave Suite 1048,Rockaway,NJ,07866
3,Tabor Road Tavern,3,4.4,https://maps.google.com/?cid=1798990059960816432,http://www.taborroadtavern.com/,510 Tabor Rd,Morris Plains,NJ,07950
4,Subway,1,3.4,https://maps.google.com/?cid=9758189877393522658,https://order.subway.com/en-us/restaurant/2197...,559 E Main St Store # 2,Denville,NJ,07834
...,...,...,...,...,...,...,...,...,...
355,Anthony & Sons Bakery & Italian Deli,2,4.5,https://maps.google.com/?cid=1646938978706323838,http://anthonyandsonsbakerynj.com/,15 NJ-10,Succasunna,NJ,07876
356,Pizza Express Mt. Arlington,1,3.7,https://maps.google.com/?cid=17474442911624320363,https://pizzaexpressmtarlington.com/,312 Howard Blvd,Mt Arlington,NJ,07856
357,Shell,2,3.1,https://maps.google.com/?cid=11162395169597474812,https://find.shell.com/us/fuel/12395193-274-rt...,274 US-46,Mine Hill Township,NJ,07803
358,Dunkin',1,4.2,https://maps.google.com/?cid=11489687353021894176,https://smart.link/abu8tr12sc1y6?utm_source=go...,274 Rte 46,Mine Hill Township,NJ,07803


# Trip Advisor Data

### Store CSV data into a DataFrame

In [267]:
csv_file = "../Resources/tripadvisor_data.csv"
tripadvisor_data_df = pd.read_csv(csv_file)
tripadvisor_data_df

,Unnamed: 0,restaurant_name,price_level,restaurant_rating,cuisine,trip_advisor_url,address
0,0,1. Gourmet Cafe Restaurant,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"136 Baldwin Rd On the Corner of Rt 46 W, Walgr..."
1,1,2. Sergio and Co Italian Specialties,$$ - $$$,5.0,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"28 Broadway, Denville, NJ 07834-2704"
2,2,3. The Quiet Man,$$ - $$$,4.5,"American, Irish",https://www.tripadvisor.com/Restaurant_Review-...,"64 E McFarlan St, Dover, NJ 07801-3522"
3,3,4. H2Ocean Restaurant & Raw Bar,$$$$,4.5,"American, Seafood",https://www.tripadvisor.com/Restaurant_Review-...,"41 Ridgedale Ave, Cedar Knolls, NJ 07927"
4,4,5. La Strada Ristorante,$$ - $$$,4.5,"Italian, European",https://www.tripadvisor.com/Restaurant_Review-...,"1105 State Route 10, Randolph, NJ 07869-1819"
...,...,...,...,...,...,...,...
1204,1204,26. Thai Kitchen,$$ - $$$,4.5,"Asian, Thai",https://www.tripadvisor.com/Restaurant_Review-...,"320 State Route 206 S, Chester, NJ 07930"
1205,1205,27. Nonna's Italian Restaurant,$$ - $$$,4.0,"Italian, Pizza",https://www.tripadvisor.com/Restaurant_Review-...,"176 Columbia Turnpike, Florham Park, NJ 07932-..."
1206,1206,28. South+Pine American Eatery,$$ - $$$,4.0,American,https://www.tripadvisor.com/Restaurant_Review-...,"90 South St, Morristown, NJ 07960-9349"
1207,1207,29. Thai Nam Phet,$$ - $$$,4.5,"Asian, Thai",https://www.tripadvisor.com/Restaurant_Review-...,"296 US Highway 46, Rockaway, NJ 07866-3852"


### Dropping duplicate restaurant address and restaurant_name value

In [268]:
# clear leading and trailing spaces
tripadvisor_data_df['restaurant_name'] = tripadvisor_data_df['restaurant_name'].str.strip()
tripadvisor_data_df['address'] = tripadvisor_data_df['address'].str.strip()

# drop duplicates for restaurant name and address
tripadvisor_data_df = tripadvisor_data_df.drop_duplicates(subset=['restaurant_name', 'address'])
tripadvisor_data_df

,Unnamed: 0,restaurant_name,price_level,restaurant_rating,cuisine,trip_advisor_url,address
0,0,1. Gourmet Cafe Restaurant,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"136 Baldwin Rd On the Corner of Rt 46 W, Walgr..."
1,1,2. Sergio and Co Italian Specialties,$$ - $$$,5.0,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"28 Broadway, Denville, NJ 07834-2704"
2,2,3. The Quiet Man,$$ - $$$,4.5,"American, Irish",https://www.tripadvisor.com/Restaurant_Review-...,"64 E McFarlan St, Dover, NJ 07801-3522"
3,3,4. H2Ocean Restaurant & Raw Bar,$$$$,4.5,"American, Seafood",https://www.tripadvisor.com/Restaurant_Review-...,"41 Ridgedale Ave, Cedar Knolls, NJ 07927"
4,4,5. La Strada Ristorante,$$ - $$$,4.5,"Italian, European",https://www.tripadvisor.com/Restaurant_Review-...,"1105 State Route 10, Randolph, NJ 07869-1819"
5,5,Parsippany's Best Pizza,$$ - $$$,4.5,"Pizza, Italian",https://www.tripadvisor.com/Restaurant_Review-...,"756 Route 46 West, Parsippany, NJ 07054"
6,6,6. Eccola,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"1082 US Highway 46, Parsippany, NJ 07054-2100"
7,7,7. Hunan Taste Chinese Restaurant,$$ - $$$,4.5,"Chinese, Asian",https://www.tripadvisor.com/Restaurant_Review-...,"67 Bloomfield Ave, Denville, NJ 07834-2728"
8,8,8. Scalini Fedeli,$$$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"63 Main St, Chatham, NJ 07928-2403"
9,9,9. Rod's Steak And Seafood Grille,$$$$,4.0,"American, Steakhouse",https://www.tripadvisor.com/Restaurant_Review-...,"1 Convent Rd The Madison Hotel, Morristown, NJ..."


In [269]:
# fill empty cells
tripadvisor_data_df = tripadvisor_data_df.fillna("")
tripadvisor_data_df

,Unnamed: 0,restaurant_name,price_level,restaurant_rating,cuisine,trip_advisor_url,address
0,0,1. Gourmet Cafe Restaurant,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"136 Baldwin Rd On the Corner of Rt 46 W, Walgr..."
1,1,2. Sergio and Co Italian Specialties,$$ - $$$,5.0,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"28 Broadway, Denville, NJ 07834-2704"
2,2,3. The Quiet Man,$$ - $$$,4.5,"American, Irish",https://www.tripadvisor.com/Restaurant_Review-...,"64 E McFarlan St, Dover, NJ 07801-3522"
3,3,4. H2Ocean Restaurant & Raw Bar,$$$$,4.5,"American, Seafood",https://www.tripadvisor.com/Restaurant_Review-...,"41 Ridgedale Ave, Cedar Knolls, NJ 07927"
4,4,5. La Strada Ristorante,$$ - $$$,4.5,"Italian, European",https://www.tripadvisor.com/Restaurant_Review-...,"1105 State Route 10, Randolph, NJ 07869-1819"
5,5,Parsippany's Best Pizza,$$ - $$$,4.5,"Pizza, Italian",https://www.tripadvisor.com/Restaurant_Review-...,"756 Route 46 West, Parsippany, NJ 07054"
6,6,6. Eccola,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"1082 US Highway 46, Parsippany, NJ 07054-2100"
7,7,7. Hunan Taste Chinese Restaurant,$$ - $$$,4.5,"Chinese, Asian",https://www.tripadvisor.com/Restaurant_Review-...,"67 Bloomfield Ave, Denville, NJ 07834-2728"
8,8,8. Scalini Fedeli,$$$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"63 Main St, Chatham, NJ 07928-2403"
9,9,9. Rod's Steak And Seafood Grille,$$$$,4.0,"American, Steakhouse",https://www.tripadvisor.com/Restaurant_Review-...,"1 Convent Rd The Madison Hotel, Morristown, NJ..."


### Split the Split address to address, city, state, zip code and country

In [270]:
# split from right with comma as delimiter
tripadvisor_data_df[['address1','city','state_zip']] = tripadvisor_data_df['address'].str.rsplit(',', 2, expand=True)
tripadvisor_data_df['state_zip'] = tripadvisor_data_df['state_zip'].str.lstrip()
tripadvisor_data_df[['state','zip code']] = tripadvisor_data_df['state_zip'].str.split(' ', 1, expand=True)
tripadvisor_data_df['zip code'] = tripadvisor_data_df['zip code'].str.split('-', expand=True)
tripadvisor_data_df

,Unnamed: 0,restaurant_name,price_level,restaurant_rating,cuisine,trip_advisor_url,address,address1,city,state_zip,state,zip code
0,0,1. Gourmet Cafe Restaurant,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"136 Baldwin Rd On the Corner of Rt 46 W, Walgr...","136 Baldwin Rd On the Corner of Rt 46 W, Walgr...",Parsippany,NJ 07054-2099,NJ,07054
1,1,2. Sergio and Co Italian Specialties,$$ - $$$,5.0,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"28 Broadway, Denville, NJ 07834-2704",28 Broadway,Denville,NJ 07834-2704,NJ,07834
2,2,3. The Quiet Man,$$ - $$$,4.5,"American, Irish",https://www.tripadvisor.com/Restaurant_Review-...,"64 E McFarlan St, Dover, NJ 07801-3522",64 E McFarlan St,Dover,NJ 07801-3522,NJ,07801
3,3,4. H2Ocean Restaurant & Raw Bar,$$$$,4.5,"American, Seafood",https://www.tripadvisor.com/Restaurant_Review-...,"41 Ridgedale Ave, Cedar Knolls, NJ 07927",41 Ridgedale Ave,Cedar Knolls,NJ 07927,NJ,07927
4,4,5. La Strada Ristorante,$$ - $$$,4.5,"Italian, European",https://www.tripadvisor.com/Restaurant_Review-...,"1105 State Route 10, Randolph, NJ 07869-1819",1105 State Route 10,Randolph,NJ 07869-1819,NJ,07869
5,5,Parsippany's Best Pizza,$$ - $$$,4.5,"Pizza, Italian",https://www.tripadvisor.com/Restaurant_Review-...,"756 Route 46 West, Parsippany, NJ 07054",756 Route 46 West,Parsippany,NJ 07054,NJ,07054
6,6,6. Eccola,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"1082 US Highway 46, Parsippany, NJ 07054-2100",1082 US Highway 46,Parsippany,NJ 07054-2100,NJ,07054
7,7,7. Hunan Taste Chinese Restaurant,$$ - $$$,4.5,"Chinese, Asian",https://www.tripadvisor.com/Restaurant_Review-...,"67 Bloomfield Ave, Denville, NJ 07834-2728",67 Bloomfield Ave,Denville,NJ 07834-2728,NJ,07834
8,8,8. Scalini Fedeli,$$$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"63 Main St, Chatham, NJ 07928-2403",63 Main St,Chatham,NJ 07928-2403,NJ,07928
9,9,9. Rod's Steak And Seafood Grille,$$$$,4.0,"American, Steakhouse",https://www.tripadvisor.com/Restaurant_Review-...,"1 Convent Rd The Madison Hotel, Morristown, NJ...",1 Convent Rd The Madison Hotel,Morristown,NJ 07960-6922,NJ,07960


### Clean Trip Advisor DataFrame to have only Zip Codes of Morris County.

In [272]:
tripadvisor_data_df = tripadvisor_data_df[tripadvisor_data_df['zip code'].isin(morris_county_zipcodes)]
tripadvisor_data_df

,Unnamed: 0,restaurant_name,price_level,restaurant_rating,cuisine,trip_advisor_url,address,address1,city,state_zip,state,zip code
0,0,1. Gourmet Cafe Restaurant,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"136 Baldwin Rd On the Corner of Rt 46 W, Walgr...","136 Baldwin Rd On the Corner of Rt 46 W, Walgr...",Parsippany,NJ 07054-2099,NJ,07054
1,1,2. Sergio and Co Italian Specialties,$$ - $$$,5.0,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"28 Broadway, Denville, NJ 07834-2704",28 Broadway,Denville,NJ 07834-2704,NJ,07834
2,2,3. The Quiet Man,$$ - $$$,4.5,"American, Irish",https://www.tripadvisor.com/Restaurant_Review-...,"64 E McFarlan St, Dover, NJ 07801-3522",64 E McFarlan St,Dover,NJ 07801-3522,NJ,07801
3,3,4. H2Ocean Restaurant & Raw Bar,$$$$,4.5,"American, Seafood",https://www.tripadvisor.com/Restaurant_Review-...,"41 Ridgedale Ave, Cedar Knolls, NJ 07927",41 Ridgedale Ave,Cedar Knolls,NJ 07927,NJ,07927
4,4,5. La Strada Ristorante,$$ - $$$,4.5,"Italian, European",https://www.tripadvisor.com/Restaurant_Review-...,"1105 State Route 10, Randolph, NJ 07869-1819",1105 State Route 10,Randolph,NJ 07869-1819,NJ,07869
5,5,Parsippany's Best Pizza,$$ - $$$,4.5,"Pizza, Italian",https://www.tripadvisor.com/Restaurant_Review-...,"756 Route 46 West, Parsippany, NJ 07054",756 Route 46 West,Parsippany,NJ 07054,NJ,07054
6,6,6. Eccola,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"1082 US Highway 46, Parsippany, NJ 07054-2100",1082 US Highway 46,Parsippany,NJ 07054-2100,NJ,07054
7,7,7. Hunan Taste Chinese Restaurant,$$ - $$$,4.5,"Chinese, Asian",https://www.tripadvisor.com/Restaurant_Review-...,"67 Bloomfield Ave, Denville, NJ 07834-2728",67 Bloomfield Ave,Denville,NJ 07834-2728,NJ,07834
8,8,8. Scalini Fedeli,$$$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"63 Main St, Chatham, NJ 07928-2403",63 Main St,Chatham,NJ 07928-2403,NJ,07928
9,9,9. Rod's Steak And Seafood Grille,$$$$,4.0,"American, Steakhouse",https://www.tripadvisor.com/Restaurant_Review-...,"1 Convent Rd The Madison Hotel, Morristown, NJ...",1 Convent Rd The Madison Hotel,Morristown,NJ 07960-6922,NJ,07960


In [273]:
# Drop unwanted columns and rename columns
tripadvisor_data_df = tripadvisor_data_df.drop(columns=['Unnamed: 0', 'address', 'state_zip'])
tripadvisor_data_df = tripadvisor_data_df.rename(columns={'restaurant_rating':'trip_advisor_rating',
                                                          'price_level':'trip_advisor_price_level',
                                                          'cuisine':'trip_advisor_cuisine'
                                                         })
tripadvisor_data_df['restaurant_name'] = tripadvisor_data_df['restaurant_name'].str.split('.').str[1]
tripadvisor_data_df['restaurant_name'] = tripadvisor_data_df['restaurant_name'].str.strip()
tripadvisor_data_df['address1'] = tripadvisor_data_df['address1'].str.strip()

tripadvisor_data_df

,restaurant_name,trip_advisor_price_level,trip_advisor_rating,trip_advisor_cuisine,trip_advisor_url,address1,city,state,zip code
0,Gourmet Cafe Restaurant,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,"136 Baldwin Rd On the Corner of Rt 46 W, Walgr...",Parsippany,NJ,07054
1,Sergio and Co Italian Specialties,$$ - $$$,5.0,Italian,https://www.tripadvisor.com/Restaurant_Review-...,28 Broadway,Denville,NJ,07834
2,The Quiet Man,$$ - $$$,4.5,"American, Irish",https://www.tripadvisor.com/Restaurant_Review-...,64 E McFarlan St,Dover,NJ,07801
3,H2Ocean Restaurant & Raw Bar,$$$$,4.5,"American, Seafood",https://www.tripadvisor.com/Restaurant_Review-...,41 Ridgedale Ave,Cedar Knolls,NJ,07927
4,La Strada Ristorante,$$ - $$$,4.5,"Italian, European",https://www.tripadvisor.com/Restaurant_Review-...,1105 State Route 10,Randolph,NJ,07869
5,NaN,$$ - $$$,4.5,"Pizza, Italian",https://www.tripadvisor.com/Restaurant_Review-...,756 Route 46 West,Parsippany,NJ,07054
6,Eccola,$$ - $$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,1082 US Highway 46,Parsippany,NJ,07054
7,Hunan Taste Chinese Restaurant,$$ - $$$,4.5,"Chinese, Asian",https://www.tripadvisor.com/Restaurant_Review-...,67 Bloomfield Ave,Denville,NJ,07834
8,Scalini Fedeli,$$$$,4.5,Italian,https://www.tripadvisor.com/Restaurant_Review-...,63 Main St,Chatham,NJ,07928
9,Rod's Steak And Seafood Grille,$$$$,4.0,"American, Steakhouse",https://www.tripadvisor.com/Restaurant_Review-...,1 Convent Rd The Madison Hotel,Morristown,NJ,07960


# Combining the Yelp and Google Data

In [274]:
# concat both yelp and google dataframes to get data from both dataframes
gy_all_df = pd.concat([yelp_df, google_data_df])
gy_all_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city,state,zip code,google_price_level,google_rating,google_maps_url
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,1181 Sussex Tpke,,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ,07869,NaN,NaN,NaN
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,500 NJ-10,sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ,07869,NaN,NaN,NaN
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,322 S Main St,4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ,07885,NaN,NaN,NaN
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,64 E Mcfarlan St,quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ,07801,NaN,NaN,NaN
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,1171 Sussex Tpke,veronarestaurant.com,$$,Italian,Randolph,NJ,07869,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Anthony & Sons Bakery & Italian Deli,NaN,NaN,15 NJ-10,http://anthonyandsonsbakerynj.com/,NaN,NaN,Succasunna,NJ,07876,2,4.5,https://maps.google.com/?cid=1646938978706323838
356,Pizza Express Mt. Arlington,NaN,NaN,312 Howard Blvd,https://pizzaexpressmtarlington.com/,NaN,NaN,Mt Arlington,NJ,07856,1,3.7,https://maps.google.com/?cid=17474442911624320363
357,Shell,NaN,NaN,274 US-46,https://find.shell.com/us/fuel/12395193-274-rt...,NaN,NaN,Mine Hill Township,NJ,07803,2,3.1,https://maps.google.com/?cid=11162395169597474812
358,Dunkin',NaN,NaN,274 Rte 46,https://smart.link/abu8tr12sc1y6?utm_source=go...,NaN,NaN,Mine Hill Township,NJ,07803,1,4.2,https://maps.google.com/?cid=11489687353021894176


In [275]:
# merge yelp and google data on inner join to get the same restaurants in both tables
gy_common_df = pd.merge(yelp_df, google_data_df, on=['restaurant_name', 'address1', 'zip code'], how='inner')
gy_common_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website_x,yelp_price_level,yelp_cuisine,city_x,state_x,zip code,google_price_level,google_rating,google_maps_url,restaurant_website_y,city_y,state_y
0,Pollos Pucalor,https://www.yelp.com/biz/pollos-pucalor-dover?...,4.5,85 E Blackwell St,pollospucalor.com,$,Latin American,Dover,NJ,07801,1,4.4,https://maps.google.com/?cid=14298467328753486970,http://www.pollospucalor.com/,Dover,NJ
1,Pub 199,https://www.yelp.com/biz/pub-199-mount-arlingt...,3.5,199 Howard Blvd,,$$,"Steakhouses, Pubs",Mount Arlington,NJ,07856,2,4.2,https://maps.google.com/?cid=6036139170178877904,https://pub199nj.com/,Mt Arlington,NJ
2,Tabor Road Tavern,https://www.yelp.com/biz/tabor-road-tavern-mor...,3.5,510 Tabor Rd,taborroadtavern.com,$$$,American (New),Morris Plains,NJ,07950,3,4.4,https://maps.google.com/?cid=1798990059960816432,http://www.taborroadtavern.com/,Morris Plains,NJ
3,Hunan Taste Chinese Restaurant,https://www.yelp.com/biz/hunan-taste-chinese-r...,4.0,67 Bloomfield Ave,hunantaste.com,$$,"Chinese, Seafood, Soup",Denville,NJ,07834,2,4.6,https://maps.google.com/?cid=3183519225252686311,http://hunantaste.com/,Denville,NJ
4,Verde Ristorante,https://www.yelp.com/biz/verde-ristorante-morr...,4.0,1012 Tabor Rd,verderistorante.com,$$,Italian,Morris Plains,NJ,07950,2,4.5,https://maps.google.com/?cid=9976836420500350003,http://verderistorante.com/,Morris Plains,NJ
5,Rockaway River Barn,https://www.yelp.com/biz/rockaway-river-barn-r...,3.0,11 Green Pond Rd,rockawayriverbarn.com,$$,"American (New), Burgers, Sports Bars",Rockaway,NJ,07866,2,4.2,https://maps.google.com/?cid=12817532522511618573,http://www.rockawayriverbarn.com/,Rockaway,NJ
6,La Cucina,https://www.yelp.com/biz/la-cucina-denville?os...,4.0,278 Diamond Spring Rd,lacucina-denville.com,$$,Italian,Denville,NJ,07834,2,4.5,https://maps.google.com/?cid=1564351430727940054,http://www.lacucina-denville.com/,Denville,NJ
7,Veggie Heaven,https://www.yelp.com/biz/veggie-heaven-denvill...,4.5,57 Bloomfield Ave,veggieheavennj.com,$$,"Chinese, Sushi Bars, Salad",Denville,NJ,07834,2,4.6,https://maps.google.com/?cid=4921425093695790070,http://www.veggieheavennj.com/,Denville,NJ
8,Fresco Mexican,https://www.yelp.com/biz/fresco-mexican-cheste...,3.5,137 Main St,frescomexican.com,$$,"Mexican, Soup",Chester,NJ,07930,2,4.4,https://maps.google.com/?cid=6452456972668682316,http://www.frescomexican.com/,Chester,NJ
9,The Grand Cafe,https://www.yelp.com/biz/the-grand-cafe-morris...,4.0,42 Washington St,thegrandcafe.com,$$$,"American (New), French, Salad",Morristown,NJ,07960,,4.4,https://maps.google.com/?cid=3279464098397647907,http://www.thegrandcafe.com/,Morristown,NJ


In [276]:
# drop duplicate restaurants based on address1 and zip code
gy_common_df = gy_common_df.drop_duplicates(subset=['restaurant_name', 'address1', 'zip code'])
gy_common_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website_x,yelp_price_level,yelp_cuisine,city_x,state_x,zip code,google_price_level,google_rating,google_maps_url,restaurant_website_y,city_y,state_y
0,Pollos Pucalor,https://www.yelp.com/biz/pollos-pucalor-dover?...,4.5,85 E Blackwell St,pollospucalor.com,$,Latin American,Dover,NJ,07801,1,4.4,https://maps.google.com/?cid=14298467328753486970,http://www.pollospucalor.com/,Dover,NJ
1,Pub 199,https://www.yelp.com/biz/pub-199-mount-arlingt...,3.5,199 Howard Blvd,,$$,"Steakhouses, Pubs",Mount Arlington,NJ,07856,2,4.2,https://maps.google.com/?cid=6036139170178877904,https://pub199nj.com/,Mt Arlington,NJ
2,Tabor Road Tavern,https://www.yelp.com/biz/tabor-road-tavern-mor...,3.5,510 Tabor Rd,taborroadtavern.com,$$$,American (New),Morris Plains,NJ,07950,3,4.4,https://maps.google.com/?cid=1798990059960816432,http://www.taborroadtavern.com/,Morris Plains,NJ
3,Hunan Taste Chinese Restaurant,https://www.yelp.com/biz/hunan-taste-chinese-r...,4.0,67 Bloomfield Ave,hunantaste.com,$$,"Chinese, Seafood, Soup",Denville,NJ,07834,2,4.6,https://maps.google.com/?cid=3183519225252686311,http://hunantaste.com/,Denville,NJ
4,Verde Ristorante,https://www.yelp.com/biz/verde-ristorante-morr...,4.0,1012 Tabor Rd,verderistorante.com,$$,Italian,Morris Plains,NJ,07950,2,4.5,https://maps.google.com/?cid=9976836420500350003,http://verderistorante.com/,Morris Plains,NJ
5,Rockaway River Barn,https://www.yelp.com/biz/rockaway-river-barn-r...,3.0,11 Green Pond Rd,rockawayriverbarn.com,$$,"American (New), Burgers, Sports Bars",Rockaway,NJ,07866,2,4.2,https://maps.google.com/?cid=12817532522511618573,http://www.rockawayriverbarn.com/,Rockaway,NJ
6,La Cucina,https://www.yelp.com/biz/la-cucina-denville?os...,4.0,278 Diamond Spring Rd,lacucina-denville.com,$$,Italian,Denville,NJ,07834,2,4.5,https://maps.google.com/?cid=1564351430727940054,http://www.lacucina-denville.com/,Denville,NJ
7,Veggie Heaven,https://www.yelp.com/biz/veggie-heaven-denvill...,4.5,57 Bloomfield Ave,veggieheavennj.com,$$,"Chinese, Sushi Bars, Salad",Denville,NJ,07834,2,4.6,https://maps.google.com/?cid=4921425093695790070,http://www.veggieheavennj.com/,Denville,NJ
8,Fresco Mexican,https://www.yelp.com/biz/fresco-mexican-cheste...,3.5,137 Main St,frescomexican.com,$$,"Mexican, Soup",Chester,NJ,07930,2,4.4,https://maps.google.com/?cid=6452456972668682316,http://www.frescomexican.com/,Chester,NJ
9,The Grand Cafe,https://www.yelp.com/biz/the-grand-cafe-morris...,4.0,42 Washington St,thegrandcafe.com,$$$,"American (New), French, Salad",Morristown,NJ,07960,,4.4,https://maps.google.com/?cid=3279464098397647907,http://www.thegrandcafe.com/,Morristown,NJ


In [277]:
# drop unwanted columns and rename rest of the columns
gy_common_df = gy_common_df.drop(columns=['restaurant_website_x','city_x','state_x'])
gy_common_df = gy_common_df.rename(columns={'restaurant_website_y':'restaurant_website',
                                                          'city_y':'city',
                                                          'state_y':'state'})
gy_common_df

,restaurant_name,yelp_url,yelp_rating,address1,yelp_price_level,yelp_cuisine,zip code,google_price_level,google_rating,google_maps_url,restaurant_website,city,state
0,Pollos Pucalor,https://www.yelp.com/biz/pollos-pucalor-dover?...,4.5,85 E Blackwell St,$,Latin American,07801,1,4.4,https://maps.google.com/?cid=14298467328753486970,http://www.pollospucalor.com/,Dover,NJ
1,Pub 199,https://www.yelp.com/biz/pub-199-mount-arlingt...,3.5,199 Howard Blvd,$$,"Steakhouses, Pubs",07856,2,4.2,https://maps.google.com/?cid=6036139170178877904,https://pub199nj.com/,Mt Arlington,NJ
2,Tabor Road Tavern,https://www.yelp.com/biz/tabor-road-tavern-mor...,3.5,510 Tabor Rd,$$$,American (New),07950,3,4.4,https://maps.google.com/?cid=1798990059960816432,http://www.taborroadtavern.com/,Morris Plains,NJ
3,Hunan Taste Chinese Restaurant,https://www.yelp.com/biz/hunan-taste-chinese-r...,4.0,67 Bloomfield Ave,$$,"Chinese, Seafood, Soup",07834,2,4.6,https://maps.google.com/?cid=3183519225252686311,http://hunantaste.com/,Denville,NJ
4,Verde Ristorante,https://www.yelp.com/biz/verde-ristorante-morr...,4.0,1012 Tabor Rd,$$,Italian,07950,2,4.5,https://maps.google.com/?cid=9976836420500350003,http://verderistorante.com/,Morris Plains,NJ
5,Rockaway River Barn,https://www.yelp.com/biz/rockaway-river-barn-r...,3.0,11 Green Pond Rd,$$,"American (New), Burgers, Sports Bars",07866,2,4.2,https://maps.google.com/?cid=12817532522511618573,http://www.rockawayriverbarn.com/,Rockaway,NJ
6,La Cucina,https://www.yelp.com/biz/la-cucina-denville?os...,4.0,278 Diamond Spring Rd,$$,Italian,07834,2,4.5,https://maps.google.com/?cid=1564351430727940054,http://www.lacucina-denville.com/,Denville,NJ
7,Veggie Heaven,https://www.yelp.com/biz/veggie-heaven-denvill...,4.5,57 Bloomfield Ave,$$,"Chinese, Sushi Bars, Salad",07834,2,4.6,https://maps.google.com/?cid=4921425093695790070,http://www.veggieheavennj.com/,Denville,NJ
8,Fresco Mexican,https://www.yelp.com/biz/fresco-mexican-cheste...,3.5,137 Main St,$$,"Mexican, Soup",07930,2,4.4,https://maps.google.com/?cid=6452456972668682316,http://www.frescomexican.com/,Chester,NJ
9,The Grand Cafe,https://www.yelp.com/biz/the-grand-cafe-morris...,4.0,42 Washington St,$$$,"American (New), French, Salad",07960,,4.4,https://maps.google.com/?cid=3279464098397647907,http://www.thegrandcafe.com/,Morristown,NJ


In [278]:
# combine the table with all the restaurants with the table with common restaurants
gy_df = pd.concat([gy_all_df, gy_common_df])
gy_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city,state,zip code,google_price_level,google_rating,google_maps_url
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,1181 Sussex Tpke,,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ,07869,NaN,NaN,NaN
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,500 NJ-10,sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ,07869,NaN,NaN,NaN
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,322 S Main St,4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ,07885,NaN,NaN,NaN
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,64 E Mcfarlan St,quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ,07801,NaN,NaN,NaN
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,1171 Sussex Tpke,veronarestaurant.com,$$,Italian,Randolph,NJ,07869,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,Anthony & Sons Bakery,https://www.yelp.com/biz/anthony-and-sons-bake...,4.0,20 Luger Rd,http://anthonyandsonsbakerynj.com/,$$,"Bakeries, Delis, Grocery",Denville,NJ,07834,2,4.5,https://maps.google.com/?cid=12111564385776009695
13,Jefferson House,https://www.yelp.com/biz/jefferson-house-lake-...,2.0,139 Nolans Point Rd,http://www.jefferson-house.com/,$$,American (Traditional),Lake Hopatcong,NJ,07849,2,3.6,https://maps.google.com/?cid=13778617519495280818
14,Olive Garden Italian Restaurant,https://www.yelp.com/biz/olive-garden-italian-...,3.0,323 Mt Hope Ave,https://www.olivegarden.com/locations/nj/rocka...,$$,"Italian, Salad, Soup",Rockaway,NJ,07866,2,4.2,https://maps.google.com/?cid=15112342085930840195
15,Mehndi,https://www.yelp.com/biz/mehndi-morristown-6?o...,4.0,3 Speedwell Ave,http://www.mehndimorristown.com/,$$,Indian,Morristown,NJ,07960,3,4.1,https://maps.google.com/?cid=3543060601759244459


## Google and Yelp Restaurant Data with unique restaurants

In [146]:
# drop the first occurance of common restaurants, last occurance contains the rows with both yelp and google data
gy_final_df = gy_df.drop_duplicates(subset=['restaurant_name', 'address1', 'zip code'], keep='last')
gy_final_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city,state,zip code,google_price_level,google_rating,google_maps_url
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,1181 Sussex Tpke,,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ,07869,NaN,NaN,NaN
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,500 NJ-10,sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ,07869,NaN,NaN,NaN
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,322 S Main St,4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ,07885,NaN,NaN,NaN
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,64 E Mcfarlan St,quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ,07801,NaN,NaN,NaN
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,1171 Sussex Tpke,veronarestaurant.com,$$,Italian,Randolph,NJ,07869,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,Anthony & Sons Bakery,https://www.yelp.com/biz/anthony-and-sons-bake...,4.0,20 Luger Rd,http://anthonyandsonsbakerynj.com/,$$,"Bakeries, Delis, Grocery",Denville,NJ,07834,2,4.5,https://maps.google.com/?cid=12111564385776009695
13,Jefferson House,https://www.yelp.com/biz/jefferson-house-lake-...,2.0,139 Nolans Point Rd,http://www.jefferson-house.com/,$$,American (Traditional),Lake Hopatcong,NJ,07849,2,3.6,https://maps.google.com/?cid=13778617519495280818
14,Olive Garden Italian Restaurant,https://www.yelp.com/biz/olive-garden-italian-...,3.0,323 Mt Hope Ave,https://www.olivegarden.com/locations/nj/rocka...,$$,"Italian, Salad, Soup",Rockaway,NJ,07866,2,4.2,https://maps.google.com/?cid=15112342085930840195
15,Mehndi,https://www.yelp.com/biz/mehndi-morristown-6?o...,4.0,3 Speedwell Ave,http://www.mehndimorristown.com/,$$,Indian,Morristown,NJ,07960,3,4.1,https://maps.google.com/?cid=3543060601759244459


# Combining the Trip Advisor with combined Yelp and Google Data

In [147]:
# concat both combined yelp and google dataframes and trip advisor data to get data from all dataframes
restaurant_all_df = pd.concat([gy_final_df, tripadvisor_data_df])
restaurant_all_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city,state,zip code,google_price_level,google_rating,google_maps_url,trip_advisor_price_level,trip_advisor_rating,trip_advisor_cuisine,trip_advisor_url
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,1181 Sussex Tpke,,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ,07869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,500 NJ-10,sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ,07869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,322 S Main St,4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ,07885,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,64 E Mcfarlan St,quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ,07801,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,1171 Sussex Tpke,veronarestaurant.com,$$,Italian,Randolph,NJ,07869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,26. Thai Kitchen,NaN,NaN,320 State Route 206 S,NaN,NaN,NaN,Chester,NJ,07930,NaN,NaN,NaN,$$ - $$$,4.5,"Asian, Thai",https://www.tripadvisor.com/Restaurant_Review-...
27,27. Nonna's Italian Restaurant,NaN,NaN,176 Columbia Turnpike,NaN,NaN,NaN,Florham Park,NJ,07932,NaN,NaN,NaN,$$ - $$$,4.0,"Italian, Pizza",https://www.tripadvisor.com/Restaurant_Review-...
28,28. South+Pine American Eatery,NaN,NaN,90 South St,NaN,NaN,NaN,Morristown,NJ,07960,NaN,NaN,NaN,$$ - $$$,4.0,American,https://www.tripadvisor.com/Restaurant_Review-...
29,29. Thai Nam Phet,NaN,NaN,296 US Highway 46,NaN,NaN,NaN,Rockaway,NJ,07866,NaN,NaN,NaN,$$ - $$$,4.5,"Asian, Thai",https://www.tripadvisor.com/Restaurant_Review-...


In [148]:
# merge yelp and google dataframes and trip advisor data on inner join to get the same restaurants in all tables
restaurant_common_df = pd.merge(gy_final_df, tripadvisor_data_df, on=['restaurant_name', 'address1', 'zip code'], how='inner')
restaurant_common_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city_x,state_x,zip code,google_price_level,google_rating,google_maps_url,trip_advisor_price_level,trip_advisor_rating,trip_advisor_cuisine,trip_advisor_url,city_y,state_y


In [149]:
# drop duplicate restaurants based on address1 and zip code
restaurant_common_df = restaurant_common_df.drop_duplicates(subset=['restaurant_name', 'address1', 'zip code'])
restaurant_common_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city_x,state_x,zip code,google_price_level,google_rating,google_maps_url,trip_advisor_price_level,trip_advisor_rating,trip_advisor_cuisine,trip_advisor_url,city_y,state_y


In [150]:
# drop unwanted columns and rename rest of the columns
restaurant_common_df = restaurant_common_df.drop(columns=['city_x','state_x'])
restaurant_common_df = restaurant_common_df.rename(columns={'restaurant_website_y':'restaurant_website',
                                                          'city_y':'city',
                                                          'state_y':'state'})
restaurant_common_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,zip code,google_price_level,google_rating,google_maps_url,trip_advisor_price_level,trip_advisor_rating,trip_advisor_cuisine,trip_advisor_url,city,state


In [151]:
# combine the table with all the restaurants with the table with common restaurants
restaurant_df = pd.concat([restaurant_all_df, restaurant_common_df])
restaurant_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city,state,zip code,google_price_level,google_rating,google_maps_url,trip_advisor_price_level,trip_advisor_rating,trip_advisor_cuisine,trip_advisor_url
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,1181 Sussex Tpke,,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ,07869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,500 NJ-10,sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ,07869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,322 S Main St,4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ,07885,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,64 E Mcfarlan St,quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ,07801,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,1171 Sussex Tpke,veronarestaurant.com,$$,Italian,Randolph,NJ,07869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,26. Thai Kitchen,NaN,NaN,320 State Route 206 S,NaN,NaN,NaN,Chester,NJ,07930,NaN,NaN,NaN,$$ - $$$,4.5,"Asian, Thai",https://www.tripadvisor.com/Restaurant_Review-...
27,27. Nonna's Italian Restaurant,NaN,NaN,176 Columbia Turnpike,NaN,NaN,NaN,Florham Park,NJ,07932,NaN,NaN,NaN,$$ - $$$,4.0,"Italian, Pizza",https://www.tripadvisor.com/Restaurant_Review-...
28,28. South+Pine American Eatery,NaN,NaN,90 South St,NaN,NaN,NaN,Morristown,NJ,07960,NaN,NaN,NaN,$$ - $$$,4.0,American,https://www.tripadvisor.com/Restaurant_Review-...
29,29. Thai Nam Phet,NaN,NaN,296 US Highway 46,NaN,NaN,NaN,Rockaway,NJ,07866,NaN,NaN,NaN,$$ - $$$,4.5,"Asian, Thai",https://www.tripadvisor.com/Restaurant_Review-...


## Google, Yelp and Trip Advisor Restaurant Data with unique restaurants

In [152]:
# drop the first occurance of common restaurants, last occurance contains the rows with both yelp and google data
restaurant_final_df = restaurant_df.drop_duplicates(subset=['restaurant_name', 'address1', 'zip code'], keep='last')

# add the resturant_id column for unique identifier
restaurant_final_df['restaurant_id'] = range(1, len(restaurant_final_df) + 1)
restaurant_final_df = restaurant_final_df.fillna("")
restaurant_final_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city,state,zip code,google_price_level,google_rating,google_maps_url,trip_advisor_price_level,trip_advisor_rating,trip_advisor_cuisine,trip_advisor_url,restaurant_id
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,1181 Sussex Tpke,,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ,07869,,,,,,,,1
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4,500 NJ-10,sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ,07869,,,,,,,,2
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,322 S Main St,4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ,07885,,,,,,,,3
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,64 E Mcfarlan St,quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ,07801,,,,,,,,4
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,1171 Sussex Tpke,veronarestaurant.com,$$,Italian,Randolph,NJ,07869,,,,,,,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,26. Thai Kitchen,,,320 State Route 206 S,,,,Chester,NJ,07930,,,,$$ - $$$,4.5,"Asian, Thai",https://www.tripadvisor.com/Restaurant_Review-...,490
27,27. Nonna's Italian Restaurant,,,176 Columbia Turnpike,,,,Florham Park,NJ,07932,,,,$$ - $$$,4,"Italian, Pizza",https://www.tripadvisor.com/Restaurant_Review-...,491
28,28. South+Pine American Eatery,,,90 South St,,,,Morristown,NJ,07960,,,,$$ - $$$,4,American,https://www.tripadvisor.com/Restaurant_Review-...,492
29,29. Thai Nam Phet,,,296 US Highway 46,,,,Rockaway,NJ,07866,,,,$$ - $$$,4.5,"Asian, Thai",https://www.tripadvisor.com/Restaurant_Review-...,493


# Separating tables

## Final Google table

In [153]:
# search for rows that have a google_maps_url
google_final_df = restaurant_final_df[restaurant_final_df.google_maps_url != ""]

# drop unwanted columns
google_final_df = google_final_df.drop(columns=['restaurant_name', 'yelp_url', 'yelp_rating', 'yelp_price_level', 'yelp_cuisine', 'trip_advisor_url', 'trip_advisor_rating', 'trip_advisor_price_level', 'trip_advisor_cuisine', 'address1', 'restaurant_website', 'city', 'state', 'zip code'])
google_final_df = google_final_df[['restaurant_id', 'google_price_level', 'google_rating', 'google_maps_url']]
google_final_df

,restaurant_id,google_price_level,google_rating,google_maps_url
0,214,1,3.7,https://maps.google.com/?cid=15487458085899557598
1,215,2,4.2,https://maps.google.com/?cid=8506076560359997359
2,216,1,4.2,https://maps.google.com/?cid=11876493694413647283
4,217,1,3.4,https://maps.google.com/?cid=9758189877393522658
5,218,2,4.2,https://maps.google.com/?cid=4442482484205730940
...,...,...,...,...
12,460,2,4.5,https://maps.google.com/?cid=12111564385776009695
13,461,2,3.6,https://maps.google.com/?cid=13778617519495280818
14,462,2,4.2,https://maps.google.com/?cid=15112342085930840195
15,463,3,4.1,https://maps.google.com/?cid=3543060601759244459


## Final Yelp table

In [155]:
# search for rows that have a yelp_url
yelp_final_df = restaurant_final_df[restaurant_final_df.yelp_url != ""]

# drop unwanted columns
yelp_final_df = yelp_final_df.drop(columns=['google_price_level', 'google_rating', 'google_maps_url', 'trip_advisor_url', 'trip_advisor_rating', 'trip_advisor_price_level', 'trip_advisor_cuisine', 'address1', 'restaurant_website', 'city', 'state', 'zip code', 'restaurant_name'])
yelp_final_df = yelp_final_df[['restaurant_id', 'yelp_price_level', 'yelp_rating', 'yelp_url', 'yelp_cuisine']]

yelp_final_df

,restaurant_id,yelp_price_level,yelp_rating,yelp_url,yelp_cuisine
0,1,$$$,4.5,https://www.yelp.com/biz/rosie-s-trattoria-ran...,"Italian, Bars, Venues & Event Spaces"
1,2,$$,4,https://www.yelp.com/biz/suburban-bar-and-kitc...,"American (New), Bars, Pizza"
2,3,$$,4.5,https://www.yelp.com/biz/4-seasons-mediterrane...,"Wine Bars, French, Italian"
3,4,$$,4.5,https://www.yelp.com/biz/quiet-man-pub-dover-2...,"Irish Pub, Seafood, Tapas/Small Plates"
4,5,$$,4.5,https://www.yelp.com/biz/verona-restaurant-ran...,Italian
...,...,...,...,...,...
12,460,$$,4,https://www.yelp.com/biz/anthony-and-sons-bake...,"Bakeries, Delis, Grocery"
13,461,$$,2,https://www.yelp.com/biz/jefferson-house-lake-...,American (Traditional)
14,462,$$,3,https://www.yelp.com/biz/olive-garden-italian-...,"Italian, Salad, Soup"
15,463,$$,4,https://www.yelp.com/biz/mehndi-morristown-6?o...,Indian


## Cuisine table from Yelp

In [156]:
# get the list of all available cuisines
cuisine_series = yelp_final_df['yelp_cuisine'].str.split(',')
cuisine_list = [item.strip() for cuisine_sublist in cuisine_series for item in cuisine_sublist]
cuisine_list = list(set(cuisine_list)) 
cuisine_list

['Salad',
 'Comfort Food',
 'Burgers',
 'Gastropubs',
 'Japanese',
 'Vietnamese',
 'Turkish',
 'Pasta Shops',
 'Korean',
 'Tapas/Small Plates',
 'Puerto Rican',
 'Beer Bar',
 'Sushi Bars',
 'Sports Bars',
 'Wraps',
 'Irish Pub',
 'Seafood',
 'Iberian',
 'Buffets',
 'Soul Food',
 'French',
 'Cheese Shops',
 'Meat Shops',
 'Chicken Shop',
 'Coffee & Tea',
 'Pizza',
 'Italian',
 'Persian/Iranian',
 'Vegan',
 'Venues & Event Spaces',
 'Acai Bowls',
 'Salvadoran',
 'Peruvian',
 'Gluten-Free',
 'Spanish',
 'Bubble Tea',
 'Restaurants',
 'Chinese',
 'Steakhouses',
 'Chicken Wings',
 'Tacos',
 'Kebab',
 'Music Venues',
 'Sandwiches',
 'Food Trucks',
 'Cheesesteaks',
 'American (Traditional)',
 'Asian Fusion',
 'Diners',
 'Desserts',
 'Latin American',
 'Irish',
 'Hot Dogs',
 'Pubs',
 'Afghan',
 'Malaysian',
 'Colombian',
 'Cocktail Bars',
 'Halal',
 'Polish',
 'Grocery',
 'Fast Food',
 'Middle Eastern',
 'Delis',
 'Breakfast & Brunch',
 'Empanadas',
 'Bakeries',
 'Portuguese',
 'Wine Bars',
 '

In [157]:
# create the cuisine table
rest_id = []
cuisine = []
for item, row in restaurant_final_df.iterrows():
    # if there is a cuisine entry
    if row['yelp_cuisine'] != "":
        # find the cuisines and add to list
        rest_cuisine = map(str.strip, row['yelp_cuisine'].split(","))
        # search for the restaurant listed cuisines in the cuisine list
        matches = set(rest_cuisine).intersection(cuisine_list)
        # add the cuisine and restaurant id to list
        for match in matches:
            cuisine.append(match)
            rest_id.append(row['restaurant_id'])

# add lists to the dataframe
cuisine_df = pd.DataFrame({'cuisine':cuisine,
                          'restaurant_id':rest_id})
# drop cuisine column from yelp data
yelp_final_df = yelp_final_df.drop(columns=['yelp_cuisine'])
cuisine_df

,cuisine,restaurant_id
0,Italian,1
1,Venues & Event Spaces,1
2,Bars,1
3,American (New),2
4,Bars,2
...,...,...
453,Soup,462
454,Indian,463
455,Coffee & Tea,464
456,Breakfast & Brunch,464


## Final Trip Advisor table

In [159]:
# search for rows that have a tripadvisor_url
ta_final_df = restaurant_final_df[restaurant_final_df.trip_advisor_url != ""]

# drop unwanted columns
ta_final_df = ta_final_df.drop(columns=['google_price_level', 'google_rating', 'google_maps_url', 'yelp_price_level', 'yelp_rating', 'yelp_url', 'yelp_cuisine', 'address1', 'restaurant_website', 'city', 'state', 'zip code', 'restaurant_name'])
ta_final_df = ta_final_df[['restaurant_id', 'trip_advisor_price_level', 'trip_advisor_rating', 'trip_advisor_url', 'trip_advisor_cuisine']]

ta_final_df

,restaurant_id,trip_advisor_price_level,trip_advisor_rating,trip_advisor_url,trip_advisor_cuisine
0,465,$$ - $$$,4.5,https://www.tripadvisor.com/Restaurant_Review-...,Italian
1,466,$$ - $$$,5,https://www.tripadvisor.com/Restaurant_Review-...,Italian
2,467,$$ - $$$,4.5,https://www.tripadvisor.com/Restaurant_Review-...,"American, Irish"
3,468,$$$$,4.5,https://www.tripadvisor.com/Restaurant_Review-...,"American, Seafood"
4,469,$$ - $$$,4.5,https://www.tripadvisor.com/Restaurant_Review-...,"Italian, European"
5,470,$$ - $$$,4.5,https://www.tripadvisor.com/Restaurant_Review-...,"Pizza, Italian"
6,471,$$ - $$$,4.5,https://www.tripadvisor.com/Restaurant_Review-...,Italian
7,472,$$ - $$$,4.5,https://www.tripadvisor.com/Restaurant_Review-...,"Chinese, Asian"
8,473,$$$$,4.5,https://www.tripadvisor.com/Restaurant_Review-...,Italian
9,474,$$$$,4,https://www.tripadvisor.com/Restaurant_Review-...,"American, Steakhouse"


### Cuisine from Trip Advisor

In [160]:
# get the list of all available cuisines
cuisine_series = ta_final_df['trip_advisor_cuisine'].str.split(',')
cuisine_list = [item.strip() for cuisine_sublist in cuisine_series for item in cuisine_sublist]
cuisine_list = list(set(cuisine_list)) 
cuisine_list

['Caribbean',
 'European',
 'Steakhouse',
 'Pizza',
 'Italian',
 'Asian',
 'Diner',
 'Bar',
 'Thai',
 'Seafood',
 'Irish',
 'Cajun & Creole',
 'Jamaican',
 'Chinese',
 'American']

In [161]:
# add trip advisor cuisines the cuisine table
for item, row in restaurant_final_df.iterrows():
    # if there is a cuisine entry
    if row['trip_advisor_cuisine'] != "":
        # find the cuisines and add to list
        rest_cuisine = map(str.strip, row['trip_advisor_cuisine'].split(","))
        # search for the restaurant listed cuisines in the cuisine list
        matches = set(rest_cuisine).intersection(cuisine_list)
        # add the cuisine and restaurant id to cuisine dataframe
        for match in matches:
            cuisine_df.append({'cuisine':match, 'restaurant_id':row['restaurant_id']},ignore_index=True,sort=False)

# drop cuisine column from yelp data
ta_final_df = ta_final_df.drop(columns=['trip_advisor_cuisine'])
cuisine_df = cuisine_df.drop_duplicates()
cuisine_df

,cuisine,restaurant_id
0,Italian,1
1,Venues & Event Spaces,1
2,Bars,1
3,American (New),2
4,Bars,2
...,...,...
453,Soup,462
454,Indian,463
455,Coffee & Tea,464
456,Breakfast & Brunch,464


## Final Restaurant table

In [162]:
# drop unwanted columns
restaurant_final_df = restaurant_final_df.drop(columns=['google_price_level', 'google_rating', 'google_maps_url', 'yelp_url', 'yelp_rating', 'yelp_price_level', 'yelp_cuisine'])
restaurant_final_df = restaurant_final_df[['restaurant_id', 'restaurant_name', 'restaurant_website', 'address1', 'city', 'state', 'zip code']]
restaurant_final_df

,restaurant_id,restaurant_name,restaurant_website,address1,city,state,zip code
0,1,Rosie’s Trattoria,,1181 Sussex Tpke,Randolph,NJ,07869
1,2,SubUrban Bar & Kitchen,sbknj.com,500 NJ-10,Randolph,NJ,07869
2,3,4 Seasons Mediterranean Restaurant,4seasonswharton.com,322 S Main St,Wharton,NJ,07885
3,4,Quiet Man Pub,quietmanpub.com,64 E Mcfarlan St,Dover,NJ,07801
4,5,Verona Restaurant,veronarestaurant.com,1171 Sussex Tpke,Randolph,NJ,07869
...,...,...,...,...,...,...,...
26,490,26. Thai Kitchen,,320 State Route 206 S,Chester,NJ,07930
27,491,27. Nonna's Italian Restaurant,,176 Columbia Turnpike,Florham Park,NJ,07932
28,492,28. South+Pine American Eatery,,90 South St,Morristown,NJ,07960
29,493,29. Thai Nam Phet,,296 US Highway 46,Rockaway,NJ,07866


In [163]:
# save to csv files
google_final_df.to_csv('../Resources/google_final_data.csv', index=False)
yelp_final_df.to_csv('../Resources/yelp_final_data.csv', index=False)
ta_final_df.to_csv('../Resources/tripadvisor_final_data.csv', index=False)
cuisine_df.to_csv('../Resources/cuisine_data.csv', index=False)
restaurant_final_df.to_csv('../Resources/restaurant_final_data.csv', index=False)